In [6]:
# import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re

In [7]:
pip install WordCloud

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Load file
jobs = pd.read_csv("UK.csv")
jobs.head()

FileNotFoundError: [Errno 2] No such file or directory: 'UK.csv'

In [ ]:
# rename the columns to suit the naming convention
jobs.rename(columns={'Job Title': 'Job_title', 'Company Score': 'Company_score', 'Date': 'Duration'}, inplace=True)
jobs.head()

In [ ]:
# Standardize company names by making all first letters in capital
jobs["Company"] = jobs["Company"].str.title()
jobs.head()

In [ ]:
# View all variations of job titles
unique_titles = jobs["Job_title"].unique()
print(unique_titles)

In [ ]:
jobs["Job_title"].value_counts().head()

In [ ]:
# There are svereal variations of the title Data Scienctist. Standardize them and make them into one
jobs['Job_title'] = jobs['Job_title'].str.replace('DATA SCIENTIST', 'Data Scientist', case=False)

In [ ]:
# To check for the total Machine Learning jobs regardless of variations
jobs["Job_title"].str.contains("Data Scientist", case=False, na=False).sum()

In [ ]:
# Machine Learning Job title variations
ml_titles = jobs[jobs["Job_title"].str.contains("Data Scientist", case=False, na=False)]
print(ml_titles["Job_title"].unique())

In [ ]:
# check for missing values
jobs.isnull().sum()

In [ ]:
# let's view all the columns with missing values
missing_values = jobs[jobs.isnull().any(axis=1)] # axis=1 refers to columns while axis=0 refers to rows
missing_values.head(5)

In [ ]:
# this counts the missing values from all the columns with missing values in terms of their rows
missing_values.count()

In [ ]:
# summary statistics
jobs.describe(include='all').head()

In [ ]:
# change the 'K' in the salary column to thousand(000)
jobs['Salary'] = jobs['Salary'].str.replace('£', '').str.replace('K', '000')
jobs.head()

In [ ]:
# replaces Employer est with an empty string
jobs['Salary'] = jobs['Salary'].str.replace(r'\(Employer est.\)', ' ', regex=True)
jobs['Salary'] = jobs['Salary'].str.replace(r'\(Glassdoor est.\)', ' ', regex=True)
jobs.head()

In [ ]:
# convert hourly salary to yearly in the salary column
def convert_to_yearly(Salary):
  if 'Per Hour' in str(Salary):
    hourly_rate = re.findall(r'\d+\.?\d*', Salary)
    if hourly_rate:
      return float(hourly_rate[0])*2080
  return Salary
jobs['Salary'] = jobs['Salary'].apply(convert_to_yearly)

In [ ]:
# split salary into minimum and maximum
jobs['Salary'] = jobs['Salary'].astype(str) # make sure the datatype is a string
jobs[['Salary_minimum', 'Salary_maximum']] = jobs['Salary'].str.split('-', expand=True)
jobs.head()

In [ ]:
jobs.info()

In [ ]:
# Define function to clean whitespace in salary columns
def clean_salary_whitespace(df, column_name):
    df[column_name] = df[column_name].str.replace(" ", "").str.strip()
    return df

# Apply function to salary columns
jobs = clean_salary_whitespace(jobs, "Salary_minimum")
jobs = clean_salary_whitespace(jobs, "Salary_maximum")

# View the updated DataFrame
jobs.head()

In [ ]:
# let's convert the Salary_max and Salary_min column to numeric
jobs['Salary_minimum'] = pd.to_numeric(jobs['Salary_minimum'], errors='coerce')
jobs['Salary_maximum'] = pd.to_numeric(jobs['Salary_maximum'], errors='coerce')
jobs.info()

In [ ]:
# convert the location datatype to string datatype in order to separate them into city and country
jobs['Location'] = jobs['Location'].astype(str)

In [ ]:
# split Location data into city and country (Make sure it's in string format)
"""
    The Location column is already in string format as converted earlier but just in case it isn't
    we convert it into a string with this code 
    jobs["Location"]=jobs["Location"].astype("string")
"""

"""
    str.split(',') - This splits the column wherever there's a comma
    n=1 - This ensures that only the first comma is used for splitting e.g, if there's a value like Ikeja, Lagos, Nigeria it only splits the 1st comma
    expand=True - Converts the result into a separate dataframe
"""
jobs[['City', 'Country']] = jobs['Location'].str.split(',', n=1, expand=True)
jobs.head()

In [ ]:
# Split the skills column
skills_split = jobs['Skills'].str.get_dummies(sep=', ')
skills_split.head()

In [ ]:
# create a new column to indicate if it is remote or onsite
jobs['Remote_Status'] = jobs['City'].apply(lambda x: 'Remote' if x == 'Remote' else 'On-site')

In [ ]:
# drop the location column
jobs.drop(columns=['Location'], inplace=True)

In [ ]:
# Replace "Remote" (case-insensitive) in the City column with NaN
jobs['City'] = jobs['City'].apply(lambda x: np.nan if isinstance(x, str) and x.strip().lower() == 'remote' else x)

In [ ]:
# Create a mask for rows where City contains 'United Kingdom' (case-insensitive)
mask = jobs['City'].str.strip().str.lower() == 'united kingdom'

# Move the country name from City to Country if the Country is currently missing
jobs.loc[mask & jobs['Country'].isnull(), 'Country'] = jobs.loc[mask & jobs['Country'].isnull(), 'City']

# Set the City value to NaN because it's not a valid city
jobs.loc[mask, 'City'] = np.nan

In [ ]:
# concatenate with the original dataframe
jobs_with_skills = pd.concat([jobs, skills_split], axis=1)
jobs_with_skills.head(5)

In [ ]:
skills_split['Python'].head()

In [ ]:
# Extract the numeric digits from the "Duration" column and convert them to int
jobs['Duration'] = pd.to_numeric(jobs['Duration'].str.extract(r'(\d+)', expand=False), errors='coerce').astype(int)

In [ ]:
# this shows us the number of jobs that didn't list (570) SQL as a skill to have and (180) listed as a skill to have
skills_split['SQL'].value_counts()

In [ ]:
# save to csv file
jobs.to_csv("UK_jobs_clean.csv", index=False)